[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fin-py/geo-data-hands-on/blob/main/docs/example.ipynb)

# 事例: 法人活動情報

ここでは、gBizINFOの職場情報から、労働者に占める女性労働者の割合や育児休業取得者数を地図に可視化し、地域ごとの特徴を確認する事例を紹介します。

> gBizINFO > 法人活動情報 データダウンロード（情報種別）
> 
> https://info.gbiz.go.jp/hojin/DownloadTop

職場情報をダウンロードし、緯度経度の列を追加します。

In [1]:
import folium
import pandas as pd
from folium.plugins import HeatMap

In [2]:
hojinjoho_coordinates = pd.read_csv(
    "https://raw.githubusercontent.com/fin-py/geo-data-hands-on/main/data/hojinjoho_coordinates.zip"
).set_index("corporate_number")
shokuba = (
    pd.read_csv("Shokubajoho_UTF-8_20231111.zip")
    .pipe(lambda df: df.loc[:, df.notna().any()])
    .set_index("法人番号")
).join(hojinjoho_coordinates)
shokuba.head()

,法人名（法人番号）,本社所在地（法人番号）,法人名,本社所在地,平均継続勤務年数-範囲,平均継続勤務年数-男性,平均継続勤務年数-女性,正社員の平均継続勤務年数,従業員の平均年齢,月平均所定外労働時間,...,育児休業対象者数（男性）,育児休業対象者数（女性）,育児休業取得者数（男性）,育児休業取得者数（女性）,postal_code,location,name,update_date,lon,lat
法人番号,,,,,,,,,,,,,,,,,,,,,
2120001030651,五洋紙工株式会社,大阪府大阪市住之江区安立４丁目１３番１８号,五洋紙工株式会社,大阪府大阪市住之江区安立４丁目１３番１８号,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5590003.0,大阪府大阪市住之江区安立４丁目１３番１８号,五洋紙工株式会社,2018-12-04T00:00:00+09:00,135.487381,34.598198
8470005000627,医療法人社団明恵会,香川県高松市多賀町２丁目５番６号,医療法人社団明恵会,香川県高松市多賀町２丁目５番６号,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7600063.0,香川県高松市多賀町２丁目５番６号,医療法人社団明恵会,2018-06-27T00:00:00+09:00,134.061234,34.337074
8120002079674,有限会社京阪動物病院,大阪府門真市垣内町１６番１６号,有限会社京阪動物病院,大阪府門真市垣内町１６番１６号,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5710065.0,大阪府門真市垣内町１６番１６号,有限会社京阪動物病院,2015-11-20T00:00:00+09:00,135.594299,34.743519
9040005001650,医療法人社団至心会,千葉県千葉市中央区新宿２丁目５番２１号,医療法人社団至心会,千葉県千葉市中央区新宿２丁目５番２１号,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2600021.0,千葉県千葉市中央区新宿２丁目５番２１号,医療法人社団至心会,2019-11-06T00:00:00+09:00,140.117004,35.607319
9180001103916,株式会社Ｒ‐ＳＴＹＬＥ,愛知県知多郡武豊町字梨子ノ木２丁目３６番地,株式会社Ｒ‐ＳＴＹＬＥ,愛知県知多郡武豊町字梨子ノ木２丁目３６番地,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4702309.0,愛知県知多郡武豊町字梨子ノ木２丁目３６番地,株式会社Ｒ‐ＳＴＹＬＥ,2018-11-09T00:00:00+09:00,136.903275,34.867893


「育児休業取得」のデータが存在する行を抽出します。

In [3]:
ikukyu = shokuba.loc[
    shokuba.loc[
        :,
        [
            "労働者に占める女性労働者の割合",
            "育児休業対象者数（男性）",
            "育児休業対象者数（女性）",
            "育児休業取得者数（男性）",
            "育児休業取得者数（女性）",
            "lat",
            "lon",
        ],
    ]
    .notnull()
    .all(axis=1),
    :,
]

「労働者に占める女性労働者の割合」をヒートマップに可視化します。

In [4]:
m = folium.Map(location=(37.8, 137), zoom_start=6, width=800, height=800)
HeatMap(data=ikukyu.loc[:, ["lat", "lon", "労働者に占める女性労働者の割合"]].values, radius=10).add_to(
    m
)
m

In [5]:
take_ikukyu_femal = ikukyu.loc[ikukyu.loc[:, "育児休業取得者数（女性）"] > 0, :]
take_ikukyu_male = ikukyu.loc[ikukyu.loc[:, "育児休業取得者数（男性）"] > 0, :]

女性の育児休業取得者数が1以上のデータをマーカーで描画します。

In [6]:
m = folium.Map(location=(37.8, 137), zoom_start=6, width=800, height=800)
HeatMap(data=ikukyu.loc[:, ["lat", "lon", "労働者に占める女性労働者の割合"]].values, radius=10).add_to(
    m
)
take_ikukyu_femal.apply(
    lambda s: folium.Marker(
        location=(s[["lat", "lon"]]), tooltip=list(s[["法人名", "育児休業取得者数（女性）"]])
    ).add_to(m),
    axis=1,
)
m

男性の育児休業取得者数が1以上のデータをマーカーで描画します。

In [7]:
m = folium.Map(location=(37.8, 137), zoom_start=6, width=800, height=800)
HeatMap(data=ikukyu.loc[:, ["lat", "lon", "労働者に占める女性労働者の割合"]].values, radius=11).add_to(
    m
)
take_ikukyu_male.apply(
    lambda s: folium.Marker(
        location=(s[["lat", "lon"]]), tooltip=list(s[["法人名", "育児休業取得者数（女性）"]])
    ).add_to(m),
    axis=1,
)
m